<a href="https://colab.research.google.com/github/a07458666/TBrainNLP/blob/master/dataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import re
import numpy as np
import pandas as pd
from google.colab import drive
from tqdm import tqdm

#crawler
import requests
from bs4 import BeautifulSoup

In [5]:
drive.mount('/content/drive')
os.chdir(r"/content/drive/My Drive/tbrainData/")
# 讀入官方的data
df = pd.read_csv('tbrain_train_final_0610.csv')  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wGHnOTEjoFVVE_wsTIZHdrWU_lrMgbl_ozmV89ZAKsAXwuBNcQRxt0
Mounted at /content/drive


In [6]:
df[0:10]

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
5,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[]
6,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[]
7,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[]
8,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[]
9,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[]


In [7]:
# 取出網站種類
webKind = df["hyperlink"].str.split('/').str.get(2)
webKind, count = np.unique(webKind, return_counts = True)
dict = {"name": webKind,  
        "counts": count
       }
webKind_df = pd.DataFrame(dict)
webKind_df
# copy df
df_addContext = df.copy()

,name,counts
0,ccc.technews.tw,10
1,domestic.judicial.gov.tw,30
2,ec.ltn.com.tw,6
3,ent.ltn.com.tw,2
4,estate.ltn.com.tw,1
5,finance.technews.tw,56
6,hk.on.cc,57
7,house.ettoday.net,1
8,m.ctee.com.tw,530
9,m.ltn.com.tw,12


In [110]:
def getUrlStatusCode(url):
  return requests.get(url).status_code

def getWebsiteCategory(url):
  return url.split('/')[2]

def getContext(url):
  category = getWebsiteCategory(url)
  if getUrlStatusCode(url) != requests.codes.ok:
        return [], []
      
  # get the news
  res = requests.get(f'{url}')
  if category in ["www.coolloud.org.tw", "news.mingpao.com", "hk.on.cc"]:
        res.encoding = 'UTF-8'
  soup = BeautifulSoup(res.text)

  # get the title
  try:
    if category in ['news.ltn.com.tw']:
      title = soup.find_all('h2').text
    else:
      title = soup.find('h1').text
  except AttributeError as e:
    print("url : ", url, "\n get title fail")
    title = []

  # get contents   
  try:
    if category in ["domestic.judicial.gov.tw"]:
      contents = soup.find_all('pre')
    elif category in ['mops.twse.com.tw']:
      contents = soup.find_all('td', attrs={'style':"text-align:left !important;"})
    elif category in ['news.ltn.com.tw']:
      contents = soup.find_all('div', attrs={"data-desc": "內容頁"})[0].find_all('p')
      print(len(contents))
    elif category in ["www.bnext.com.tw"]:
      try:
          contents = soup.find_all('div', attrs={"data-url": url})[0].find_all('p')
      except:
          contents = soup.find_all('p')
    else:
      contents = soup.find_all('p')
  except:
    print("url : ", url, "\n get contents fail")
    return title, []

  lines = []
  for content in contents:
    try:
      content.find('a').txet
      continue
    except:
      pass
    oneLine = re.sub('[\(\)\{\}<>\n\t\xa0\r\u3000]', '', content.text)  
    lines.append(content.text)
  article = "".join(lines).strip()
  return title, article

In [111]:
# 單獨測試
# 有問題 5
index = 100
url = df["hyperlink"][index]
category = getWebsiteCategory(url)
title, article = getContext(url) 
print("url :", url, "\ncategory :", category,'\ntitle', title, "\narticle :", article)
if article is not None:
  df_addContext.loc[index, "title"] = str(title)
  df_addContext.loc[index, "article"] = str(article)

11
url : http://news.ltn.com.tw/news/society/breakingnews/2875648 
category : news.ltn.com.tw 
title 湯景華縱火6死案 律師團要求鑑定「再犯可能性」被圍剿 
article : 高院更一審今提訊縱火害死6命的湯景華，媒體問湯，陳述案情記憶力很好，怎麼會主張無就審能力？湯不作答。（記者楊國文攝）〔記者楊國文／台北報導〕52歲男子湯景華，被控和翁姓男子有傷害官司，但因對方無罪竟挾怨報復，3年前凌晨到翁的新北市三重住處縱火洩恨，造成翁家6人葬生火窟，一、二審均依殺人罪判湯死刑，最高法院發回更審，湯男多次主張他無就審能力，去過縱火現場但否認犯罪，但更一審鑑定湯有就審能力，行為時也有辨識違法能力，今庭訊湯的律師又指名，應就再犯可能性，找之前幫八里雙屍案被告謝依涵鑑定的台大心理系教授趙儀珊鑑定，卻遭檢方和告訴人律師圍剿，均指稱無再鑑定必要。高檢署檢察官曾文鐘直指，此案已做過心理衡鑑，「今天審理湯景華縱火害死6人案，不是不符合辯護人意思，就要求爺爺、告奶奶，非得到律師團要的結論不可吧！」而且，趙儀珊的專長，是在性侵害方面的鑑定，與此案不吻合，曾認為，最高法院發回希望就湯景華的「再犯可能性」、「再社會化」兩部份詳查，監所教化應是很重要的一環，但湯男在所表現很不好，一直和人吵架，顯示自我控制能力差，而北捷殺人魔鄭捷殺害4人，湯害死6人，「難道還要讓社會承擔後果？」，曾強調「一個不知認錯的人，毫無教化可能性」。請繼續往下閱讀...告訴人律師也指出，湯景華蓄意殺人和審理中否認犯罪的表現，顯示無教化、矯治的可能性，湯的律師團從頭到尾指名替謝依涵擔任再犯可能性的鑑定人趙儀珊教授擔任其鑑定人，「充其量只是在拖延訴訟」。謝依涵最後逆轉改判無期徒刑，逃過一死。不過，湯的律師團指出，湯景華涉及6死命案，不能因此省略應進行有無再犯、教化可能性的必要鑑定，而趙儀珊教授曾在法院進行過多案的鑑定，是適當的鑑定人選。高院合議庭諭知，湯的律師團應於兩天內，述明建議由趙儀珊教授擔任心理衡鑑鑑定人的事由，呈報給合議庭做為准、駁的依據。今日庭訊尾聲，高院合議庭也召開延押庭，檢方主張有逃亡之虞，應延押，湯的律師請給湯具保停押；合議庭近日將書面裁定。湯景華2014年4月到三重一家小吃店用餐，期間抱怨服務態度差，且不滿老闆娘

In [112]:
#df_addContext['contextCrawler'] = df_addContext['hyperlink'].apply(lambda x: getContext(x))
min = 0
max = 99999
hyperlinkList = df_addContext['hyperlink']
for index, url in enumerate(hyperlinkList):
    if index > max or index < min:
      continue
    category = getWebsiteCategory(url)
    title, article = getContext(url)
    print("\n========index", index, "============\nurl :", url, "\ncategory :", category,'\ntitle', title, "\narticle :", article)
    if article is not None:
      df_addContext.loc[index, "title"] = str(title)
      df_addContext.loc[index, "article"] = str(article)
    else:
      print('{url} content is None')

串流輸出內容已截斷至最後 5000 行。
========index 4506 ============
url : https://udn.com/news/story/7238/3961260 
category : news.ltn.com.tw 
title [] 
article : 

========index 4507 ============
url : https://www.mirrormedia.mg/story/20191119soc008/ 
category : news.ltn.com.tw 
title 青山宮遶境回宮順序延後　轎班等不及爆衝突 
article : 全部類別
    
      全部類別
    
    全文朗讀
  
    00:00 / 00:00台北市萬華區宮廟青山宮，本月16日展開為期3天的遶境活動，吸引許多信徒擺香壇迎駕，卻在活動最後一日主轎回鑾，陣頭排隊入廟行禮時，3名男子竟爆發推擠，二十多名警力也緊急趕赴現場控制，讓活動順利結束。警方調查，今（19日）凌晨1時50分許，隊伍抵達西昌街與貴陽街二段口時，有廟方及各陣頭共3位民眾因神轎入廟時間延誤，導致進場順序不同，因而產生口角、爆發衝突。警方立刻啟動快打警力，到場後立即強勢控制現場，現場無人受傷，為避免衝突擴大，立即分開雙方人員，並第一時間介入協調廟方與各陣頭秩序後，衝突未再發生，繞境隊伍也按時順利入廟。萬華分局表示，艋舺青山宮連續3天的廟會，分局事前即非常重視，並增加警力派遣全力維持治安以及交通秩序。希望信眾勿因一時誤會讓活動變了調，也感謝這3天民眾及各位警察同仁的配合，讓遶境活動平安、順利落幕。至於3名動手打架的人，警方也將傳喚到案，並依社維法依法送辦。
        更新時間｜2019.11.21 17:51弟外借機車惹禍　兄滿街找車殺死少年後自首「蔡賴配」合體　易經命理老師：得票數上看715萬票相關關鍵字：

========index 4508 ============
url : https://m.ctee.com.tw/livenews/ch/20190702002357-260405 
category : news.ltn.com.tw 
title    工商時報 - 您投資理財的好夥伴   
article : 108學年

In [113]:
df_addContext.to_csv('train_final.csv')